In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
import cv2
import numpy as np
from sklearn import svm
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction

import multiprocessing as mp
from tqdm import tqdm

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# Binary feather files
train_f = pd.read_feather("../data/binary/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f = pd.read_feather("../data/binary/val_all_features.f", columns=None, use_threads=True, storage_options=None)
train_shape = train_f.shape
val_shape = val_f.shape
print(f'train shape: {train_shape}, val shape: {val_shape}')
# Multi-class feather files.
train_f_mul = pd.read_feather("../data/three_class/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f_mul = pd.read_feather("../data/three_class/val_all_feat.f", columns=None, use_threads=True, storage_options=None)
train_shape_m = train_f_mul.shape
val_shape_m = val_f_mul.shape
print(f'train shape multi: {train_shape_m}, val shape multi: {val_shape_m}')

train shape: (15195, 144), val shape: (3796, 144)
train shape multi: (5082, 144), val shape multi: (1270, 144)


In [4]:
## Pre-processing
# Binary case
train_f.replace({'nevus': 1, 'others': 0}, inplace=True)
val_f.replace({'nevus': 1, 'others': 0}, inplace=True)

df_train = train_f.iloc[:train_shape[0], :].sample(frac=1, random_state=42)
df_val = val_f.iloc[:val_shape[0], :].sample(frac=1, random_state=42)
print(f"df_train: {df_train.shape}, df_val: {df_val.shape}")

X_train, y_train = df_train.iloc[:, :(train_shape[1]-1)].to_numpy(dtype=np.float32), df_train.iloc[:, (train_shape[1]-1)].to_numpy()
X_test, y_test = df_val.iloc[:, :(val_shape[1]-1)].to_numpy(dtype= np.float32), df_val.iloc[:, (val_shape[1]-1)].to_numpy()

print(f'X_train_new: {X_train.dtype}, y_train: {y_train.shape}, X_test_new: {X_test.shape}, y_test: {y_test.shape}')

X_train_f = pd.DataFrame(X_train)
X_test_f = pd.DataFrame(X_test)
y_train_f = pd.DataFrame(y_train)
y_test_f = pd.DataFrame(y_test)
frames = [X_train_f, X_test_f]
f = [y_train_f, y_test_f]
X = pd.concat(frames)
y = pd.concat(f)

# Feature Selection -- VARIANCE THRESHOLD
sel_var = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_var= sel_var.fit_transform(X)
print(f'X_var: {X_var.shape}')

## Feature Selection -- SelectFromModel
sel_sfm = SelectFromModel(LinearSVC(C = 0.09, penalty="l1",  dual=False))
print("Performing Feature Selection ... ")
X_sfm = sel_sfm.fit_transform(X, y)
print(f'X_sfm: {X_sfm.shape}')

# split dataframe X again into train and test -- VARIANCE THRESHOLD
X_train_var = pd.DataFrame(X_var).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_var = pd.DataFrame(X_var).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_var: {X_train_var.shape}, y_train: {y_train.shape}, X_test_var: {X_test_var.shape}, y_test: {y_test.shape}')

# split dataframe X again into train and test SFM
X_train_new = pd.DataFrame(X_sfm).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_new = pd.DataFrame(X_sfm).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_new: {X_train_new.shape}, y_train: {y_train.shape}, X_test_new: {X_test_new.shape}, y_test: {y_test.shape}')

df_train: (15195, 144), df_val: (3796, 144)
X_train_new: float32, y_train: (15195,), X_test_new: (3796, 143), y_test: (3796,)
X_var: (18991, 91)
Performing Feature Selection ... 


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X_sfm: (18991, 86)
X_train_var: (15195, 91), y_train: (15195,), X_test_var: (3796, 91), y_test: (3796,)
X_train_new: (15195, 86), y_train: (15195,), X_test_new: (3796, 86), y_test: (3796,)


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# Random Forest without feature selection
param_grid = { 'classifier__max_features': ['auto', 'sqrt'],
 'classifier__n_estimators': [50,100,200,400,800],
'classifier__class_weight': ["balanced", None]}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
classifier = RandomForestClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
grid_search = GridSearchCV(pipe, param_grid, cv=cv, refit = True, verbose = 3)
grid_search.fit(X_train, y_train)
print("TRAIN- The best parameters are %s with an accuracy of %0.4f"%(grid_search.best_params_, grid_search.best_score_))
y_train_predicted = grid_search.predict(X_train)
y_test_predicted =  grid_search.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

Fitting 10 folds for each of 20 candidates, totalling 200 fits


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 1/10] END classifier__class_weight=balanced, classifier__max_features=auto, classifier__n_estimators=50;, score=0.811 total time=  19.8s


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 2/10] END classifier__class_weight=balanced, classifier__max_features=auto, classifier__n_estimators=50;, score=0.812 total time=  17.4s


...

[CV 8/10] END classifier__class_weight=balanced, classifier__max_features=auto, classifier__n_estimators=800;, score=0.815 total time= 3.0min


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 9/10] END classifier__class_weight=balanced, classifier__max_features=auto, classifier__n_estimators=800;, score=0.816 total time= 2.9min


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 10/10] END classifier__class_weight=balanced, classifier__max_features=auto, classifier__n_estimators=800;, score=0.822 total time= 3.0min
[CV 1/10] END classifier__class_weight=balanced, classifier__max_features=sqrt, classifier__n_estimators=50;, score=0.814 total time=  10.9s
[CV 2/10] END classifier__class_weight=balanced, classifier__max_features=sqrt, classifier__n_estimators=50;, score=0.815 total time=  10.8s
[CV 3/10] END classifier__class_weight=balanced, classifier__max_features=sqrt, classifier__n_estimators=50;, score=0.826 total time=  11.3s
[CV 4/10] END classifier__class_weight=balanced, classifier__max_features=sqrt, classifier__n_estimators=50;, score=0.819 total time=  11.0s
[CV 5/10] END classifier__class_weight=balanced, classifier__max_features=sqrt, classifier__n_estimators=50;, score=0.821 total time=  11.1s
[CV 6/10] END classifier__class_weight=balanced, classifier__max_features=sqrt, classifier__n_estimators=50;, score=0.829 total time=  11.3s
[CV 7/10] E

In [5]:
# Random Forest without feature selection
param_grid = { 'classifier__max_features': ['auto', 'sqrt'],
 'classifier__n_estimators': [200,400,800],
'classifier__class_weight': ["balanced", None]}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
classifier = RandomForestClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
grid_search = GridSearchCV(pipe, param_grid, cv=cv, refit = True, verbose = 3)
grid_search.fit(X_train, y_train)
print("TRAIN- The best parameters are %s with an accuracy of %0.4f"%(grid_search.best_params_, grid_search.best_score_))
y_train_predicted = grid_search.predict(X_train)
y_test_predicted =  grid_search.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU


Fitting 10 folds for each of 12 candidates, totalling 120 fits


INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU


[CV 1/10] END classifier__class_weight=balanced, classifier__max_features=auto, classifier__n_estimators=200;, score=0.817 total time=  11.3s


INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU
INFO:root:sklearn.ensemble.RandomForestClassifier.fit: running accelerated version on CPU


...

[CV 10/10] END classifier__class_weight=None, classifier__max_features=sqrt, classifier__n_estimators=800;, score=0.819 total time=  35.7s


INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU


TRAIN- The best parameters are {'classifier__class_weight': 'balanced', 'classifier__max_features': 'sqrt', 'classifier__n_estimators': 400} with an accuracy of 0.8256


INFO:root:sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU


F1 Score of train data: 1.0000 Acc: 1.0
F1 Score of test data: 0.8249 Acc: 0.8250790305584826
